Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

/home/fja0568/src/brownbear
/home/fja0568/src/brownbear/universe/sp500-galaxy


In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# read in sp500.csv
sp500 = pd.read_csv('sp500.csv')
sp500.drop(columns=['SEC filings', 'Headquarters Location', 'Date first added','CIK', 'Founded'], inplace=True)
sp500.rename(columns={'Security':'Description',
                      'GICS Sector':'Asset Class'}, inplace=True)
sp500.set_index("Symbol", inplace=True)
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M Company,Industrials,Industrial Conglomerates
ABT,Abbott Laboratories,Health Care,Health Care Equipment
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals
ABMD,ABIOMED Inc,Health Care,Health Care Equipment
ACN,Accenture plc,Information Technology,IT Consulting & Other Services
...,...,...,...
YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments
ZBH,Zimmer Biomet,Health Care,Health Care Equipment


In [6]:
# read in gics-2-asset-class.csv
gics2asset_class = pd.read_csv('gics-2-asset-class.csv', skip_blank_lines=True, comment='#')
gics2asset_class.set_index("GICS", inplace=True)
gics2asset_class = gics2asset_class['Asset Class'].to_dict()
gics2asset_class

{'Energy': 'US Stocks:Energy',
 'Materials': 'US Stocks:Materials',
 'Industrials': 'US Stocks:Industrials',
 'Consumer Discretionary': 'US Stocks:Consumer Discretionary',
 'Consumer Staples': 'US Stocks:Consumer Staples',
 'Health Care': 'US Stocks:Healthcare',
 'Financials': 'US Stocks:Financials',
 'Information Technology': 'US Stocks:Technology',
 'Communication Services': 'US Stocks:Communication Services',
 'Utilities': 'US Stocks:Utilities',
 'Real Estate': 'US Stocks:Real Estate'}

In [7]:
# map sp500 GICS sectors to brownbear defined asset classes
def _asset_class(row):
    return gics2asset_class[row['Asset Class']]

sp500['Asset Class'] = sp500.apply(_asset_class, axis=1)

# yahoo finance uses '-' where '.' is used in symbol names
sp500.index = sp500.index.str.replace('.', '-')
sp500

,Description,Asset Class,GICS Sub-Industry
Symbol,,,
MMM,3M Company,US Stocks:Industrials,Industrial Conglomerates
ABT,Abbott Laboratories,US Stocks:Healthcare,Health Care Equipment
ABBV,AbbVie Inc.,US Stocks:Healthcare,Pharmaceuticals
ABMD,ABIOMED Inc,US Stocks:Healthcare,Health Care Equipment
ACN,Accenture plc,US Stocks:Technology,IT Consulting & Other Services
...,...,...,...
YUM,Yum! Brands Inc,US Stocks:Consumer Discretionary,Restaurants
ZBRA,Zebra Technologies,US Stocks:Technology,Electronic Equipment & Instruments
ZBH,Zimmer Biomet,US Stocks:Healthcare,Health Care Equipment


In [8]:
# make symbols list
symbols = list(sp500.index)
#symbols

In [9]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

.........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................


In [10]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df = df[:]
df.tail()[['NVDA', 'WST', 'ADBE', 'NOW', 'AMD', 'PYPL', 'AAPL', 'MSFT', 'DHR', 'TMO', 'AMZN', 'DXCM', 'ODFL', 'REGN', 'ROL', 'FDX', 'TMUS', 'CDNS', 'CRM', 'MSCI']]

,NVDA,WST,ADBE,NOW,AMD,PYPL,AAPL,MSFT,DHR,TMO,AMZN,DXCM,ODFL,REGN,ROL,FDX,TMUS,CDNS,CRM,MSCI
Date,,,,,,,,,,,,,,,,,,,,
2020-11-30,535.90,275.16,478.47,534.55,92.66,214.12,119.05,214.07,224.63,464.98,3168.04,319.68,203.21,516.03,57.18,286.58,132.94,116.30,245.80,409.42
2020-12-01,535.44,272.01,479.15,539.59,92.63,216.54,122.72,216.21,225.00,471.10,3220.08,327.72,211.04,514.07,57.54,287.76,133.73,115.65,241.35,414.50
2020-12-02,541.62,275.39,481.26,526.52,93.74,212.56,123.08,215.37,224.80,478.89,3203.53,333.20,204.83,505.13,55.16,291.72,133.14,116.43,220.78,416.16
2020-12-03,535.84,272.24,484.28,525.24,92.31,214.54,122.94,214.24,223.34,477.20,3186.73,344.63,207.01,498.84,55.15,292.70,131.19,116.68,220.97,419.30
2020-12-04,542.33,271.55,486.00,525.70,94.04,217.77,122.25,214.36,224.43,474.62,3162.58,348.03,209.25,492.45,55.55,294.88,131.80,118.34,225.86,421.71


In [11]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns['MMM']

8.82862336113217

In [12]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [13]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola['MMM']

0.20913230374939587

In [14]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola['MMM']

0.10253705304570353

In [15]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly['MMM'])

Date
2015-01-31   136.72
2015-02-28   142.95
2015-03-31   139.82
2015-04-30   132.56
2015-05-31   135.69
2015-06-30   131.62
2015-07-31   129.09
2015-08-31   122.09
2015-09-30   121.77
2015-10-31   135.03
2015-11-30   135.37
2015-12-31   130.24
2016-01-31   130.55
2016-02-29   136.60
2016-03-31   145.10
2016-04-30   145.76
2016-05-31   147.55
2016-06-30   153.51
2016-07-31   156.35
2016-08-31   158.10
2016-09-30   155.45
2016-10-31   145.81
2016-11-30   152.46
2016-12-31   158.52
2017-01-31   155.19
2017-02-28   166.51
2017-03-31   170.95
2017-04-30   174.98
2017-05-31   183.79
2017-06-30   187.13
2017-07-31   180.82
2017-08-31   184.71
2017-09-30   189.75
2017-10-31   208.09
2017-11-30   220.91
2017-12-31   213.85
2018-01-31   227.60
2018-02-28   215.24
2018-03-31   200.63
2018-04-30   177.66
2018-05-31   181.49
2018-06-30   181.02
2018-07-31   195.37
2018-08-31   195.39
2018-09-30   195.19
2018-10-31   176.25
2018-11-30   193.92
2018-12-31   177.71
2019-01-31   186.81
2019-02-28   19

In [16]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns['MMM']

Date
2015-01-31     nan
2015-02-28    0.05
2015-03-31   -0.02
2015-04-30   -0.05
2015-05-31    0.02
              ... 
2020-08-31    0.09
2020-09-30   -0.02
2020-10-31   -0.00
2020-11-30    0.09
2020-12-31   -0.00
Freq: M, Name: MMM, Length: 72, dtype: float64

In [17]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev['MMM']

0.23065284848148046

In [18]:
# read investment-options-header.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
lines

['# Description: S&P 500 investment options. 09/29/2020',
 '',
 '# Format',
 '"Investment Option","Description","Asset Class","1 mo","3 mo","1 Yr","3 Yr","5 Yr","Vola","DS Vola","Std Dev"',
 '# Note: "Description" field is optional']

In [19]:
# for each symbol, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = lines.copy()

# This is still slow (2.53 s)
for i, (index, row) in enumerate(sp500.iterrows()):

    symbol = index
    description = row['Description']
    asset_class = row['Asset Class']

    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append(
        '"{}","{}","{}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
        .format(symbol, description, asset_class,
                ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [20]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')